In [ ]:
# Full clear function
def clear_all():
    globals().clear()
    import gc, tensorflow as tf
    tf.keras.backend.clear_session()
    gc.collect()

clear_all()


In [ ]:
              #Rename Dataset,(Set: 350 x 350), Augmentation - Apply

# 📦 STEP 1: প্রয়োজনীয় লাইব্রেরি ইমপোর্ট
import os
import zipfile
from PIL import Image
import shutil

# 📁 STEP 2: Google Colab-এ ফাইল আপলোড
from google.colab import files
uploaded = files.upload()  # এখানে WilDFire.zip আপলোড করো

# 📂 STEP 3: Zip ফাইল আনজিপ করো
zip_filename = "WilDFire.zip"
extract_dir = "WildFire_Extracted"

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# ✅ STEP 4: Image রিনেম ও রিসাইজ
input_dir = os.path.join(extract_dir, os.listdir(extract_dir)[0])  # WilDFire ফোল্ডার
output_dir = "WildFire_Processed_Images"
os.makedirs(output_dir, exist_ok=True)

image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
count = 1

for filename in sorted(os.listdir(input_dir)):
    file_path = os.path.join(input_dir, filename)
    ext = os.path.splitext(filename)[1].lower()

    if ext in image_extensions:
        try:
            with Image.open(file_path) as img:
                img = img.convert('RGB')  # RGB তে কনভার্ট
                img = img.resize((350, 350))  # রিসাইজ ৩৫০ x ৩৫০
                new_filename = f"{count}.jpg"
                img.save(os.path.join(output_dir, new_filename))
                count += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")

print("✅ Done! Total images processed:", count - 1)

# 🗜️ STEP 5: Processed ছবিগুলো zip করা
shutil.make_archive("WildFire_Processed_Images", 'zip', output_dir)

# ⬇️ STEP 6: ডাউনলোড লিংক তৈরি
files.download("WildFire_Processed_Images.zip")


In [ ]:
# STEP 1: Upload zip
from google.colab import files
uploaded = files.upload()

# STEP 2: Unzip
import zipfile, os

zip_filename = next(iter(uploaded))
extract_path = "/content/Satellite_Image_Detection"

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped successfully!")

# STEP 3: Dynamically locate correct folder names
def find_subfolder(parent, target_name):
    for name in os.listdir(parent):
        if name.lower() == target_name.lower():
            return os.path.join(parent, name)
    return None

base_path = os.path.join(extract_path, "Image_data(satellite)")
wildfire_path = find_subfolder(base_path, "WildFire")
nonwildfire_path = find_subfolder(base_path, "NonWildFire")

print("🔥 WildFire Path:", wildfire_path or "❌ Not found")
print("🌿 NonWildFire Path:", nonwildfire_path or "❌ Not found")

# STEP 4: Image Processing
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

IMG_SIZE = 350
data = []

valid_extensions = ['.jpg', '.jpeg', '.png']

def process_images(folder_path, label):
    if not folder_path or not os.path.exists(folder_path):
        print(f"🚫 Folder not found: {folder_path}")
        return
    files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[-1].lower() in valid_extensions]
    valid = 0
    for img_name in tqdm(files, desc=f"Processing {'WildFire' if label==1 else 'NonWildFire'}"):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Skipped unreadable: {img_path}")
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        data.append((img_path, label))
        valid += 1
    print(f"✅ Valid images: {valid}/{len(files)}")

# STEP 5: Process Images
process_images(wildfire_path, 1)
process_images(nonwildfire_path, 0)

# STEP 6: Balance + Save CSV
if len(data) == 0:
    print("🚫 No valid images found. CSV not generated.")
else:
    wildfire_data = [d for d in data if d[1] == 1]
    nonwildfire_data = [d for d in data if d[1] == 0]
    min_len = min(len(wildfire_data), len(nonwildfire_data))

    if min_len == 0:
        print("🚫 Cannot balance dataset — one class has 0 images.")
    else:
        balanced_data = wildfire_data[:min_len] + nonwildfire_data[:min_len]
        random.shuffle(balanced_data)

        df = pd.DataFrame(balanced_data, columns=["image_path", "label"])
        csv_path = "/content/fire_labels.csv"
        df.to_csv(csv_path, index=False)

        print(f"✅ CSV created with {len(df)} rows: {csv_path}")
        files.download(csv_path)


Saving Image_data(satellite).zip to Image_data(satellite).zip
✅ Unzipped successfully!
🔥 WildFire Path: /content/Satellite_Image_Detection/Image_data(satellite)/WilDFire
🌿 NonWildFire Path: /content/Satellite_Image_Detection/Image_data(satellite)/NonWildFire


Processing WildFire: 100%|██████████| 1500/1500 [00:01<00:00, 880.13it/s]


✅ Valid images: 1500/1500


Processing NonWildFire: 100%|██████████| 1500/1500 [00:02<00:00, 568.71it/s]


✅ Valid images: 1500/1500
✅ CSV created with 3000 rows: /content/fire_labels.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# STEP 1: .zip এবং .csv ফাইল আপলোড করুন
from google.colab import files
uploaded = files.upload()  # Image_data(satellite).zip এবং Update Fire labels.csv সিলেক্ট করুন

# STEP 2: .zip ফাইল আনজিপ করুন
import zipfile
import os

zip_filename = [f for f in uploaded if f.endswith('.zip')][0]
csv_filename = [f for f in uploaded if f.endswith('.csv')][0]

extract_path = "/content/extracted_data"
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ আনজিপ সম্পন্ন:", extract_path)

# STEP 3: .csv লোড এবং ডেটা শাফল
import pandas as pd

df = pd.read_csv(csv_filename)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle the data

# STEP 4: ডেটা ভাগ করা (train, val, test)
train_size = int(0.7 * len(df))  # 70% for train
val_size = int(0.15 * len(df))  # 15% for validation
test_size = len(df) - train_size - val_size  # Remaining 15% for test

train_df = df[:train_size]
val_df = df[train_size:train_size+val_size]
test_df = df[train_size+val_size:]

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# STEP 5: টার্গেট ফোল্ডার স্ট্রাকচার তৈরি
import shutil

base_output = "/content/dataset_split"
splits = {'train': train_df, 'val': val_df, 'test': test_df}
labels_map = {0: 'NonWildFire', 1: 'WildFire'}

# Create necessary directories and move images
for split, split_df in splits.items():
    for label in [0, 1]:
        dir_path = os.path.join(base_output, split, labels_map[label])
        os.makedirs(dir_path, exist_ok=True)

        # Filter the data for the given label
        subset = split_df[split_df['label'] == label]
        for i, row in subset.iterrows():
            src_path = os.path.join(extract_path, "Image_data(satellite)", row['image_path'])
            dst_path = os.path.join(dir_path, os.path.basename(row['image_path']))
            try:
                shutil.copy2(src_path, dst_path)
            except Exception as e:
                print(f"❌ ফাইল কপি করা যায়নি: {src_path} → {e}")

print("✅ ফোল্ডার তৈরি ও ডেটা কপি সম্পন্ন!")

# STEP 6: .zip করে ডাউনলোডযোগ্য করা
output_zip_path = "/content/dataset_split.zip"
shutil.make_archive("/content/dataset_split", 'zip', base_output)

# STEP 7: ডাউনলোড করুন
files.download(output_zip_path)


Saving FIRE_labels.csv to FIRE_labels.csv
Saving Image_data(satellite).zip to Image_data(satellite).zip
✅ আনজিপ সম্পন্ন: /content/extracted_data
Train: 2100 | Val: 450 | Test: 450
✅ ফোল্ডার তৈরি ও ডেটা কপি সম্পন্ন!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>